<img src="Figures/top_ML.png" alt="Drawing" style="width: 1100px;"/>

# EJERCICIO
# Aprendizaje no supervisado: Clustering.

## *Clustering de consumidores*


En el aprendizaje no supervisado, la tarea clásica es el **análisis de clusters** (grupos) en el que se encuentran patrones o grupos ocultos en los datos. La mayoría de las veces las tareas de aprendizaje no supervisado tienen una *solución abierta*, por lo que hay que interpretar los resultados y comprobar si tienen sentido.

**Objetivo:** En este ejemplo se utilizan datos que contienen información acerca del consumo eléctrico de un grupo de consumidores eléctricos. El objetivo es encontrar el número óptimo de clusters para agrupar los diferentes patrones de consumo diarios. El resultado se utilizará para fines comerciales y estratégicos.

### Antes de empezar:

* En el archivo **clustering-consumos.xlsx** se encuentra el conjunto de datos de entrada de este ejemplo (atributos). 
* **NO** existen las etiquetas en el Aprendizaje **NO Supervisado**. 


<img src="Figures/no-supervisado.png" alt="Drawing" style="width: 600px;"/>


## **1. Importar librerías y datos**

In [ ]:
# Importar librerías
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Librería de visualización de datos
plt.style.use('seaborn')

# Seleccionamos las columnas que necesitamos
df_consumos = pd.read_excel('Data\S7-Clustering-consumos.xlsx')
df_consumos

## **2. Comprender los datos**

Es necesario visualizar y comprender los datos con los que vamos a trabajar, así como conocer sus características. 

<div class="alert alert-success">
    <b> ¿Cuántos datos hay?¿Cuántos atributos hay en los datos? </b>
</div>

In [ ]:
# Dimensión de los datos de entrada (filas x columnas)
df_consumos.shape

In [ ]:
# Veamos como es la apariencia de los datos
df_consumos.head()

In [ ]:
df_consumos.tail()

<div class="alert alert-success">
    <b> Ponemos como índice el identificador del contador inteligente (CUP) </b>
</div>

In [ ]:
# Pongo como índice el número de CUP 
df_consumos.set_index('CUPs', inplace = True)
df_consumos

<div class="alert alert-success">
    <b> Comprobamos si existe algún dato categórico que haya que transformar </b>
</div>

In [ ]:
df_consumos.dtypes

<div class="alert alert-success">
    <b> ¿Falta algún dato? </b>
</div>

Se comprueba si falta algún dato, y de ser así, se realiza el recuento de celdas vacías en cada atributo. En este caso, no falta ningún dato en el conjunto de datos de entrada (no existen valores *Nan*).

In [ ]:
df_consumos.isna().sum()

Aplicamos **interpolación** para imputar los valores que faltan

In [ ]:
# Imputación de datos con pandas
df_consumos.interpolate(method='polynomial', order=1, inplace = True)


In [ ]:
# Comprobamos que se han imputado los valores correctamente
df_consumos.isna().sum()

<div class="alert alert-success">
    <b> Resumen estadístico del conjunto de datos de entrada: </b>
</div>

La estadística descriptiva recolecta y analiza el conjunto de datos de entrada con el objetivo de describir las características y comportamientos de este conjunto mediante las siguientes medidas resumen: número total de observaciones (count), media (mean), desviación estándar (std), valor mínimo (min), valor máximo (max) y los valores de los diferentes cuartiles (25%, 50%, 75%).

In [ ]:
# Evaluamos la naturaleza de los datos con datos estadísticos descriptivos
df_consumos.describe()

## **3. Visualizar los datos**

Una manera visual de entender los datos de entrada. 
1. Histograma
2. Curva de densidad
3. Boxplots


<div class="alert alert-success">
    <b>Histograma </b>
</div>


Respresentación gráfica de cada uno de los atributos en forma de barras, donde la superficie de la barra es proporcional a la frecuencia de los valores representados.

In [ ]:
histograma = df_consumos.hist(xlabelsize=10, ylabelsize=10, bins=100, figsize=(18, 12))

<div class="alert alert-success">
    <b> Gráfico de densidades </b>
</div>

Visualiza la distribución de los datos. Es una variable del histograma, pero elimina el ruido, por lo que son mejores para determinar la forma de distribución de un atributo. Lo spicos del gráfico de densidad ayudan a mostrar dónde los valores se concentran más. 

In [ ]:
density = df_consumos.plot(kind='kde', legend=True, layout=(1, 1), figsize=(18, 12),
                        fontsize=20, stacked=True) 

<div class="alert alert-success">
    <b> Boxplots </b>
</div>


El boxplot (diagrama de caja) nos permite identificar los valores atípicos y comparar distribuciones. Además, se conoce como se distribuyen el 50% de los valores (dentro de la caja).
 

In [ ]:
boxplot = df_consumos.plot(kind='box', legend=True, layout=(1, 1), figsize=(18, 12),
                        fontsize=16, stacked=True) 


## *4. Preparar los datos*



<div class="alert alert-success">
    <b> Graficamos los datos de consumo </b>
</div>

El gráfico muestra el consumo horario de un grupo de consumidores durante un día.

In [ ]:
# Consumo horario
df_consumos.T.plot(figsize=(18, 8), title='Consumo diario', legend=False, color='blue', alpha=0.05, 
                   fontsize=15, xlim=[0,23], ylabel='Energía Consumida [kWh]')



<div class="alert alert-success">
    <b> Escalar los datos </b>
</div>


<img src="Figures\scaling.png" alt="Drawing" style="width: 400px;"/>

Como ya se ha comentado, tanto **MinMaxScaler()** como **StandardScaler()** se utilizan comúnmente para escalar datos antes de aplicar el algoritmo de clustering. Sin embargo, la elección entre ellos depende de las características específicas de los datos y requisitos del análisis.

* **MinMaxScaler()** escala los datos a un rango fijo, generalmente entre 0 y 1. Esto puede ser útil si los datos tienen un rango limitado y se desea preservar la relación entre los valores de diferentes features/características. Sin embargo, puede no ser adecuado para datos con valores atípicos/outliers, ya que pueden tener un impacto desproporcionado en la escala.

* **StandardScaler()** escala los datos para tener una media cero y una varianza unitaria, lo que lo hace útil para datos que están distribuidos de manera normal o tienen una distribución similar. Puede ser más robusto a los valores atípicos que MinMaxScaler(), pero puede que no preserve la relación entre los valores de diferentes características.

Se escalan los datos utilizando el método de *MinMaxScaler()*

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = df_consumos.values.copy()
X_scale = pd.DataFrame(scaler.fit_transform(X))
X_scale.head()


<div class="alert alert-success">
    <b> Guardamos el scaler de los datos de entreno para utilizarlo luego </b>
</div>


In [ ]:
import joblib

# Guardamos el scaler en un archivo para utilizarlo luego
joblib.dump(scaler, 'scaler.joblib')

## 5. Construcción del modelo de aprendizaje NO supervisado: Clustering de consumos utilizando K-means

Se agrupan los datos utilizando el algoritmo [K-Means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

El algoritmo K-means necesita que se le indique el número de clústers en que se quieren agrupar los datos. Se ejecuta el algoritmo para varios clusters y luego se comparan los resultados utilizando el método Elbow, que indicará el número óptimo de clusters.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

elbow_method = []

# Evalúo el algoritmo K-means para un rango de [2,10] clústers 
n_cluster_list = range(2,15)
print(list(n_cluster_list))

### ¿Cómo saber el número óptimo de clusters? Con el método de Elbow.



<div class="alert alert-success">
    <b> Aplicamos el método de Elbow como métrica para selecionar un número óptimo de clusters. 
</div>
    
Se utiliza el [Método de Elbow] para ayudarnos a elegir el número óptimo de clusters. 

* Este método utiliza los valores de la inercia obtenidos tras aplicar el K-means a diferente número de Clusters (desde 1 a N Clusters), siendo la inercia la suma de las distancias al cuadrado de cada objeto del Cluster a su centroide.
* Para hacer uso de este método partimos del cálculo de la distorsión promedio de cada clúster, esto es la distancia de cada elemento con su centroide correspondiente.
* Buscamos la parte de la gráfica donde la línea es menos suave o cambia abruptamente lo que forma un “codo”.

[Método de Elbow]: https://jarroba.com/seleccion-del-numero-optimo-clusters/

    
**EJEMPLO:**

<img src="Figures\elbow-method.png" alt="Drawing" style="width: 800px;"/>

<div class="alert alert-success">
    <b> Método de Elbow. 
</div>

In [ ]:
import matplotlib.pyplot as plt

# Iteración para evaluar K-means para diferentes números de clusters (n_clusters)
for n_cluster in n_cluster_list:
    kmeans = KMeans(n_clusters=n_cluster, random_state=0)
    cluster_found = kmeans.fit_predict(X_scale)
    elbow_method.append(kmeans.inertia_) 


# Gráfica del método de Elbow
plt.plot(n_cluster_list, elbow_method, 'bx-')
plt.xlabel('Número de centroides')
plt.ylabel('Within-Cluster Sum of Square')
plt.title('Método Elbow para número óptimo de clusters')
plt.show()

# El número óptimo de clusters es...¿

<div class="alert alert-success">
    <b> Entrenar el algorithmo de clustering K-Means con k clusters 
</div>

In [ ]:
# Entreno el K-means para k=X, visto el resultado del método Elbow
kmeans = KMeans(n_clusters=4)
cluster_found = kmeans.fit_predict(X_scale)
cluster_found_sr = pd.Series(cluster_found, name='cluster')

# Creo un multindex del tipo: (fecha,cluster al que pertenece el día)
df_consumos = df_consumos.set_index(cluster_found_sr, append=True)

#Guardamos los clusters en un excel
df_consumos.to_excel('Data/S7-resultados-clusters.xlsx')

df_consumos.index

<div class="alert alert-success">
    <b> Mostrar los resultados de este Clustering de consumos 
</div>

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14,5))
color_list = ['blue', 'green', 'red', 'orange']
cluster_values = sorted(df_consumos.index.get_level_values('cluster').unique())

for cluster, color in zip(cluster_values, color_list):
    # ploteo todas las lineas de cada cluster
    df_consumos.xs(cluster, level=1).T.plot(ax=ax, legend=False, alpha=0.05, color=color)
    # ploteo la línea con el valor de la mediana de cada cluster
    df_consumos.xs(cluster, level=1).median().plot(ax=ax, color=color, legend=False, alpha=1, ls='--')

ax.set_ylabel('Potencia media horaria [kW]')
ax.set_xlabel('Horas')


## (Extra) Validar los resultados con Dimensionality Reduction (PCA)

*  Explicación de PCA visualmente. https://setosa.io/ev/principal-component-analysis/


* Principal Component Analysis (PCA) es un método estadístico que permite simplificar la complejidad de espacios muestrales con muchas dimensiones a la vez que conserva su información. Se reducen las "features" de 24 a 2. 
* Una forma de validar los resultados del algoritmo clustering es mediante técnicas de dimensionality reduction. Hay que tener en cuenta es que el PCA no sabe nada de los grupos encontrados por K-means.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.colors

pca = PCA(n_components=2)  
results_pca = pca.fit_transform(X_scale)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list(cluster_values, color_list)

plt.scatter(results_pca[:, 0], results_pca[:, 1],
            c = df_consumos.index.get_level_values('cluster'),
            cmap=cmap,
            alpha=0.4,
            )
plt.show()

<div class="alert alert-success">
    <b> Guarda el modelo entrenado de K-means para utilizarlo más tarde y no tener que entrenarlo de nuevo. </b>
</div>


<img src="Figures\save-ml-model.png" alt="Drawing" style="width: 1200px;"/>


In [ ]:

import joblib

# Save the model to a file
joblib.dump(kmeans, 'kmeans_model.joblib')

# Cargar un modelo de K-Means guardado y predecir clusters a partir de nuevos datos.

<div class="alert alert-success">
    <b> Carga el modelo entrenado de K-means y el scaler con los datos de entreno. </b>
</div>


In [ ]:
# Cargo el modelo de k-means
kmeans = joblib.load('kmeans_model.joblib')
kmeans

In [ ]:
# Cargo el scaler del archivo
scaler = joblib.load('scaler.joblib')
scaler

<div class="alert alert-success">
    <b> Cargo los nuevos datos de entrada </b>
</div>

In [ ]:
import pandas as pd

new_data = pd.read_excel('Data/S7-clustering-consumos-new-data.xlsx')

In [ ]:

# Pongo como índice el número de CUP 
new_data.set_index('CUPs', inplace = True)
new_data.head()

<div class="alert alert-success">
    <b> Debo de escalar estos nuevos datos de entrada, ya que en el entrenamiento han sido escalados. </b>
</div>


In [ ]:

new_data_scaled = pd.DataFrame(scaler.transform(new_data))
#new_data_scaled.columns = new_data.columns
new_data_scaled.head()

In [ ]:
# !pip install --upgrade scikit-learn threadpoolctl

<div class="alert alert-success">
    <b> Realizo predicciones de cluster de los datos nuevos, utilizando el modelo K-Means entrenado anteriormente. </b>
</div>

In [ ]:

predicted_labels = kmeans.predict(new_data_scaled)

In [ ]:
# Visualizo el resultado del clustering de datos nuevos
predicted_labels

In [ ]:
cluster_found_new_data_sr = pd.Series(predicted_labels, name='cluster')
cluster_found_new_data_sr

In [ ]:
# Creo un multindex del tipo: (fecha,cluster al que pertenece el día)
new_data = new_data.set_index(cluster_found_new_data_sr, append=True)
new_data.index

<div class="alert alert-success">
    <b> RECORDATORIO: Grafico otra vez los consumos de entreno obtenidos, para luego comparar. </b>
</div>


In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(14,5))
color_list = ['blue', 'green', 'red', 'orange']
cluster_values = sorted(df_consumos.index.get_level_values('cluster').unique())

for cluster, color in zip(cluster_values, color_list):
    # ploteo todas las lineas de cada cluster
    df_consumos.xs(cluster, level=1).T.plot(ax=ax, legend=False, alpha=0.05, color=color)
    # ploteo la línea con el valor de la mediana de cada cluster
    df_consumos.xs(cluster, level=1).median().plot(ax=ax, color=color, legend=False, alpha=1, ls='--')
    print("Color:", color, "se asocia al clúster ", cluster)

ax.set_ylabel('Potencia media horaria [kW]')
ax.set_xlabel('Horas')


<div class="alert alert-success">
    <b> Grafico los nuevos datos </b>
</div>

In [ ]:
# Consumo horario
new_data.T.plot(figsize=(14, 5), title='Consumo diario', legend=False, color='blue', alpha=0.5, 
                   fontsize=15, xlim=[0,23], ylabel='Energía Consumida [kWh]')

<div class="alert alert-success">
    <b> Unifico las dos gráficas y coloreo acorde al cluster asociado </b>
</div>

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14,5))
color_list = ['blue', 'green', 'red', 'orange']
cluster_values = sorted(df_consumos.index.get_level_values('cluster').unique())

print("CLUSTERS entrenamiento")
for cluster, color in zip(cluster_values, color_list):
    # ploteo todas las lineas de cada cluster
    df_consumos.xs(cluster, level=1).T.plot(ax=ax, legend=False, alpha=0.05, color=color)
    # ploteo la línea con el valor de la mediana de cada cluster
    df_consumos.xs(cluster, level=1).median().plot(ax=ax, color=color, legend=False, alpha=1, ls='--')
    print("Color:", color, "se asocia al clúster ", cluster)

#new data
print("CLUSTERS nuevos datos de entrada")
color_list = ['blue', 'red', 'orange']
cluster_values = sorted(new_data.index.get_level_values('cluster').unique())
for cluster, color in zip(cluster_values, color_list):
    # ploteo todas las lineas de cada cluster
    new_data.xs(cluster, level=1).T.plot(ax=ax, legend=False, alpha=0.40, color=color)
    # ploteo la línea con el valor de la mediana de cada cluster
    #new_data.xs(cluster, level=1).median().plot(ax=ax, color=color, legend=False, alpha=1, ls='--')   
    print("Color:", color, "se asocia al clúster ", cluster)

    
ax.set_ylabel('Potencia media horaria [kW]')
ax.set_xlabel('Horas')


<div class="alert alert-success">
    <b> Muestro solo los consumos del nuevo dataset, coloreados según su cluster asociado según K-Means </b>
</div>

In [ ]:
#new data
fig, ax = plt.subplots(1, 1, figsize=(14,5))
print("CLUSTERS nuevos datos de entrada")
color_list = ['blue', 'red', 'orange']
cluster_values = sorted(new_data.index.get_level_values('cluster').unique())
for cluster, color in zip(cluster_values, color_list):
    # ploteo todas las lineas de cada cluster
    new_data.xs(cluster, level=1).T.plot(ax=ax, legend=False, alpha=0.40, color=color)
    # ploteo la línea con el valor de la mediana de cada cluster
    #new_data.xs(cluster, level=1).median().plot(ax=ax, color=color, legend=False, alpha=1, ls='--')   
    print("Color:", color, "se asocia al clúster ", cluster)

    
ax.set_ylabel('Potencia media horaria [kW]')
ax.set_xlabel('Horas')